In [57]:
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_ai21 import ChatAI21
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_tool_calling_agent, AgentExecutor
from tools import search_tool


In [58]:
load_dotenv()

True

In [59]:
llm = ChatAI21(
    model="jamba-large"
)

In [60]:
response = llm.invoke("what is the capital of France?")

In [61]:
print(response)

content='The capital of France is **Paris**.' additional_kwargs={} response_metadata={} id='run-de120314-4ef0-4805-8ace-c4a84e9569e1-0'


In [62]:
class ResearchResponse(BaseModel):
    topic: str
    summary: str
    sources: list[str]
    tools_used: list[str]

In [63]:
parser = PydanticOutputParser(pydantic_object=ResearchResponse)

In [81]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", 
         """
         You are a research assistant that will help generate a research paper
         Answer the query and use necessary tools
         Wrap the output in this format and provide no other text\n{format_instructions}
         """,
         ),
        ("placeholder","{chat_history}"),
        ("human","{query}"),
        ("placeholder","{agent_scratchpad}"),
    ]
).partial(format_instructions=parser.get_format_instructions())

In [82]:
tools = [search_tool]

In [83]:
agent = create_tool_calling_agent(
    llm=llm,
    prompt=prompt,
    tools=tools,
)

In [84]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [85]:
try:
    raw_response = agent_executor.invoke({
        "query": "How did france achieved independence?",
        "chat_history": []
    })
    print("Raw response:", raw_response)
    
    if raw_response.get("output"):
        structured_response = parser.parse(raw_response["output"])
        print("Structured response:", structured_response)
    else:
        print("No output received from agent")
except Exception as e:
    print(f"Error occurred: {str(e)}")



> Entering new AgentExecutor chain...


> Finished chain.
Raw response: {'query': 'How did france achieved independence?', 'chat_history': [], 'output': ''}
No output received from agent


In [73]:
print(raw_response)

{'query': 'How did france achieved independence?', 'chat_history': [], 'output': ''}


In [52]:
structured_response = parser.parse(raw_response.get("output"))


In [53]:
print(structured_response)

topic='How did France achieve independence?' summary='France did not need to achieve independence as it was never colonized. Instead, it was a colonial power itself, controlling vast territories across the globe. However, France underwent significant political and social transformations, such as the French Revolution (1789-1799), which led to the establishment of a republic and the end of monarchical rule.' sources=['https://www.history.com/topics/france/french-revolution', 'https://www.britannica.com/place/France/The-French-Revolution'] tools_used=['Wikipedia', 'Google Scholar']


In [56]:
try:
    structured_response = parser.parse(raw_response.get("output"))
except Exception as e:
    print(f"Error parsing response: {e}")
    